In [ ]:
In essence, oceanLevelParameter is a user-defined parameter that controls the threshold at which a point is considered part 
of the beach or ocean. By adjusting this parameter, you can control the level of the virtual ocean in the simulation. 
A higher value of oceanLevelParameter will result in a higher beach level, while a lower value will result in a lower beach 
level. This parameter allows the simulation to be tailored to different scenarios or landscapes.

In [ ]:
Initial Conditions:
Grid and Topography:

NX and NY define the number of rows and columns in the grid.
d sets the grid spacing in meters, and dx and dy are set accordingly.
LX and LY define the dimensions of the landscape based on the grid spacing and the number of rows/columns.
Random small features are added to the topography to initiate erosion.

Physical Parameters:

K is a parameter related to the erosional process.
D represents a diffusion coefficient.
uplift denotes the rate of vertical uplift.

Model Parameters:

dt is the time step used in the simulation.
m and n are exponents in the erosion equation.
theta_c is an erosion threshold.
T is the total simulation time.

Simulation:
The LandscapeWithOcean class is initialized with NX and NY.

oceanLevelParameter is set, which controls the initial ocean level. This parameter determines how high the beach level is set relative to the minimum and maximum elevations in the landscape.

The ComputeOceanVolumeFromOceanLevelParameter method calculates the initial ocean volume based on the provided oceanLevelParameter.

pool_check is called to identify pools, drains, and drainage points in the landscape.

The simulation loop iterates through time steps. For each time step:

Collection areas are calculated using calculate_collection_area.
The landscape evolution equation is solved using erosion, diffusion, and uplift equations.
Boundary conditions and stability checks are applied.
Pools, drains, and drainage points are updated.
The simulation updates the figures periodically to visualize the evolving landscape.

Interpretation:
    
Initial Conditions:

The initial conditions are chosen by generating a random topography with small hills.
These hills represent random geological features that can influence erosion and landscape evolution.

Simulation Evolution:

Running the code multiple times yields different landscape evolutions due to the random nature of the initial conditions.
Over time, the landscape tends to smooth out, with hills eroding and valleys filling in.

Geological Features:

Initially, the landscape may have small hills and valleys.
As the simulation progresses, erosional processes smooth out sharp features, resulting in a more gradual and uniform terrain.

Ocean Dynamics:

The ocean level is initially set using the oceanLevelParameter.
As the simulation progresses, the ocean level may change based on the evolving landscape.


In summary, the code simulates the evolution of a landscape, taking into account erosion, diffusion, uplift, and ocean 
dynamics. The initial conditions represent a random topography with small geological features, which are then modified over 
time. The ocean level may also change as the landscape evolves.

In [ ]:
Describe what geological features you were trying to model even if their representation is not perfect.

I was trying to model multiple hills as my new geological features. 

Briefly describe how your landscape evolved and identify effects that are missing from your simulation if there are any.

In the animation, the hills in my simulation are simulating erosion by become shorter and are seen to be "melting".

In my code, after changing the parameters, it can be seen that the landscape changed, along with the erosion time and
the time it took for this erosion to be seen.

In [6]:
%matplotlib tk

import numpy as np
from landscapeWithOcean import LandscapeWithOcean

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import ffmpeg

def AddHill(Z,NX,NY,xx,yy,r,h):
    for x in range(NX):
        for y in range(NY):
            dx = np.mod(x-xx+NX/2,NX)-NX/2; 
            dy = np.mod(y-yy+NY/2,NY)-NY/2;
            dr = np.sqrt(dx**2+dy**2);
            if (dr<r):
                Z[x,y] += h * (np.cos(dr/r*np.pi/2.0))**2;
    return Z

### Define simulation grid and initial conditions

NX = 140 # Increased number of rows
NY = 140 # Increased number of columns

d  = 5  # grid spacing in meters
dx = d  
dy = d

LX = NX*dx
LY = NY*dy

# Define custom landscape
Z = np.zeros((NX, NY))

# Add custom features
Z = AddHill(Z, NX, NY, NX/4, NY/4, 20, 20)  # Large hill
Z = AddHill(Z, NX, NY, 3*NX/4, 3*NY/4, 15, 10)  # Smaller hill
Z = AddHill(Z, NX, NY, NX/2, NY/2, 10, -15)  # Valley

x = np.arange(NX)
y = np.arange(NY)
X, Y = np.meshgrid(y, x)
ZMaxOrg = np.max(Z)

### Physical Parameters
K = 5.0e-6  # Increased erosion coefficient
D = 0.01   # Increased diffusion coefficient

# Exponents
n = 2  # Increased sensitivity to steep gradients
m = 1.5  # Increased weight on larger cells

### Ocean Level
oceanLevelParameter = 0.05  # Lower ocean level

# Time step
dt = 90  # Smaller time step

# Initialize landscape 
ls = LandscapeWithOcean(NX,NY)
ls.ComputeOceanVolumeFromOceanLevelParameter(Z, NX, NY, oceanLevelParameter)
ls.pool_check(Z, NX, NY)
ls.A = np.zeros((NX,NY))

# Set-up figure
def init_figure():
    fig = plt.figure(figsize=(12.,6.))
    plt.show()
    return fig

def update_figure():
    plt.clf()
    ax1 = fig.add_subplot(121,projection='3d')

    # use equal x-y aspect with an explicit vertical exaggeration
    vert_exag = 4.
    ax1.set_xlim3d(0, max(NX, NY))
    ax1.set_ylim3d(0, max(NX, NY))
    ax1.set_zlim3d(0, ZMaxOrg)

    ax1.set_title('Surface Relief')

    ZPlot = np.copy(Z)
    ZPlot[ZPlot < ls.ZBeachLevel] = ls.ZBeachLevel 
    ZPlot -= ls.ZBeachLevel
    ax1.plot_surface(X, Y, ZPlot, cmap=cm.terrain, rstride=1, cstride=1, antialiased=False, linewidth=0)

    ax2 = fig.add_subplot(122, aspect='equal')
    ax2.set_title('Elevation')

    im = ax2.pcolor(ZPlot, cmap=cm.coolwarm)
    cs = ax2.contour(ZPlot, 6, colors='k')

    cbar = fig.colorbar(im, shrink=0.5, aspect=5)
    cbar.add_lines(cs)

    plt.draw()
    plt.pause(0.0001)

# Set up figure
fig = init_figure()
update_figure()
Znew = np.copy(Z)

n_iter = 200  # Increased number of iterations

for it in range(1, n_iter+1):
    
    ls.calculate_collection_area(Z, NX, NY)
    ls.A *= dx*dy
    
    for i in range(NX):
        iL = np.mod(i-1, NX)
        iR = np.mod(i+1, NX)

        for j in range(NY):
            jD = np.mod(j-1, NY)
            jU = np.mod(j+1, NY)
  
            if ls.ocean[i, j] > 0:
                Psi_z = 0
                Phi_z = 0
            else:
                if ls.drain[i, j] > 0:
                    s1 = (Z[iR, j] - Z[iL, j]) / (2.0 * dx)
                    s2 = (Z[i, jU] - Z[i, jD]) / (2.0 * dy)
                    s3 = (Z[iR, jD] - Z[iL, jU]) / (2.0 * np.sqrt(dx**2 + dy**2))
                    s4 = (Z[iR, jU] - Z[iL, jD]) / (2.0 * np.sqrt(dx**2 + dy**2))
                    gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2)) / 2.0
                    Psi_z = K * (ls.A[i, j]**m * gradient**n - theta_c)            

                elif ls.drainage[i, j] > 0:
                    if (Z[i, j] >= Z[iR, j]) and ls.pool[iR, j] != ls.drainage[i, j]: 
                        gradient = (Z[i, j] - Z[iR, j]) / dx
                    elif (Z[i, j] >= Z[iL, j]) and ls.pool[iL, j] != ls.drainage[i, j]:
                        gradient = (Z[i, j] - Z[iL, j]) / dx
                    elif (Z[i, j] >= Z[i, jU]) and ls.pool[i, jU] != ls.drainage[i, j]:
                        gradient = (Z[i, j] - Z[i, jU]) / dy
                    elif (Z[i, j] >= Z[i, jD]) and ls.pool[i, jD] != ls.drainage[i, j]:
                        gradient = (Z[i, j] - Z[i, jD]) / dy
                    else:
                        gradient = 0.02
                    Psi_z = K * (ls.A[i, j]**m * gradient**n - theta_c)
                
                else:
                    Psi_z = 0
                
                if (Psi_z < 0):
                    Psi_z = 0 

                Phi_z = D * ((Z[iR, j] - 2.0*Z[i, j] + Z[iL, j]) / dx**2 + (Z[i, jU] - 2.0*Z[i, j] + Z[i, jD]) / dy**2 )

            Znew[i, j] = Z[i, j] + (Phi_z - Psi_z + uplift) * dt  

            dZdt = (Znew[i, j] - Z[i, j]) / dt
            CFL = abs(dZdt) * dt / min(dx, dy)
            if (CFL > 1.0):
                print('\nWarning: Time step of', dt, 'is probably too large. Safer would be:', dt/CFL)
            
            if (Znew[i, j] < 0.):
                Znew[i, j] = 0.
    
    Z = np.copy(Znew)
    ls.pool_check(Z, NX, NY)

    if (np.mod(it, 10) == 0): 
        print(it, end='')
        update_figure()
        print(' Ocean level=', ls.ZBeachLevel, ' Ocean surface fraction=', 100*ls.AOcean/(NX*NY));
    else:
        print('.', end='')

update_figure()
print(' Simulation finished.')

Minimum elevation           -15.0
Maximum elevation           20.0
Beach level                 -13.25
Ocean volume                12.639709209341966
Percentage of ocean surface 0.0663265306122449





























.........10 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........20 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........30 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........40 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........50 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........60 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........70 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........80 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........90 Ocean level= 0.000762999168302316  Ocean surface fraction= 

KeyboardInterrupt: 

In [4]:
%matplotlib tk 
import numpy as np
from landscapeWithOcean import LandscapeWithOcean 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import ffmpeg

def AddHill(Z, NX, NY, xx, yy, r, h):
    for x in range(NX):
        for y in range(NY):
            dx = np.mod(x-xx+NX/2, NX)-NX/2
            dy = np.mod(y-yy+NY/2, NY)-NY/2
            dr = np.sqrt(dx**2+dy**2)
            if (dr<r):
                Z[x, y] += h * (np.cos(dr/r*np.pi/2.0))**2
    return Z

### Define simulation grid and initial conditions

NX = 70*2
NY = 70*2
d  = 5 
dx = d 
dy = d

LX = NX*dx
LY = NY*dy

Z = np.zeros((NX, NY))

# Create a custom landscape
Z = AddHill(Z, NX, NY, NX/4, NY/4, 30, 20)
Z = AddHill(Z, NX, NY, 3*NX/4, 3*NY/4, 30, 15)
Z = AddHill(Z, NX, NY, NX/4, 3*NY/4, 20, 10)
Z = AddHill(Z, NX, NY, 3*NX/4, NY/4, 20, 10)

x = np.arange(NX)
y = np.arange(NY)
X, Y = np.meshgrid(y, x)
ZMaxOrg = np.max(Z)

### Physical Parameters
K = 1.0e-6
D = 0.005
uplift = 0.0

### Model parameters
dt = 90 
m = 1
n = 1
theta_c = 10 
T = 1000.0 * 625.0
n_iter = int(np.round(T/dt))

### Initialize landscape 
ls = LandscapeWithOcean(NX, NY)
oceanLevelParameter = 0.1
ls.ComputeOceanVolumeFromOceanLevelParameter(Z, NX, NY, oceanLevelParameter)
ls.pool_check(Z, NX, NY)
ls.A = np.zeros((NX, NY))

def init_figure():
    fig = plt.figure(figsize=(12.,6.))
    plt.show()
    return fig

def update_figure():
    plt.clf()
    ax1 = fig.add_subplot(121, projection='3d')
    vert_exag = 4.
    ax1.set_xlim3d(0, max(NX, NY))
    ax1.set_ylim3d(0, max(NX, NY))
    ax1.set_zlim3d(0, ZMaxOrg)

    ZPlot = np.copy(Z)
    ZPlot[ZPlot<ls.ZBeachLevel] = ls.ZBeachLevel 
    ZPlot -= ls.ZBeachLevel
    ax1.plot_surface(X, Y, ZPlot, cmap=cm.terrain, rstride=1, cstride=1, antialiased=False, linewidth=0)

    ax2 = fig.add_subplot(122, aspect='equal')
    ax2.set_title('Elevation')
    im = ax2.pcolor(ZPlot, cmap=cm.coolwarm)
    cs = ax2.contour(ZPlot, 6, colors='k')
    cbar = fig.colorbar(im, shrink=0.5, aspect=5)
    cbar.add_lines(cs)
    plt.draw()
    plt.pause(0.0001)

# Set up figure
fig = init_figure()
update_figure()
Znew = np.copy(Z)

n_iter = 200 

for it in range(1, n_iter+1):
    ls.calculate_collection_area(Z, NX, NY)
    ls.A *= dx*dy
    
    for i in range(NX):
        iL = np.mod(i-1, NX)
        iR = np.mod(i+1, NX)

        for j in range(NY):
            jD = np.mod(j-1, NY)
            jU = np.mod(j+1, NY)
  
            if ls.ocean[i, j] > 0:
                Psi_z = 0
                Phi_z = 0
            else:
                if ls.drain[i, j] > 0:
                    s1 = (Z[iR, j] - Z[iL, j])/(2.*dx)
                    s2 = (Z[i, jU] - Z[i, jD])/(2.*dy)
                    s3 = (Z[iR, jD] - Z[iL, jU])/(2. * np.sqrt(dx**2 + dy**2) )
                    s4 = (Z[iR, jU] - Z[iL, jD])/(2. * np.sqrt(dx**2 + dy**2) )
                    gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2))/2.
                    Psi_z = K * ( ls.A[i, j]**m * gradient**n - theta_c)            
                # ... [Rest of the code for erosion simulation]

    Znew[0,:] = 0.0 
    Z = np.copy(Znew)
    ls.pool_check(Z, NX, NY)

    if (np.mod(it, 10) == 0): 
        print(it, end='')
        update_figure()
        print(' Ocean level=', ls.ZBeachLevel, ' Ocean surface fraction=', 100*ls.AOcean/(NX*NY));
    else:
        print('.', end='')

update_figure()
print(' Simulation finished.')

 dt[years] =  90
Number of interation:  6944
Minimum elevation           0.3757057741436256
Maximum elevation           100.0
Beach level                 10.338135196729263
Ocean volume                1180.024649282962
Percentage of ocean surface 1.2244897959183674

....
.....10 Ocean level= 10.348707713532797  Ocean surface fraction= 1.2653061224489797
.........20 Ocean level= 10.348707713532797  Ocean surface fraction= 1.2653061224489797
.........30 Ocean level= 10.345750146174538  Ocean surface fraction= 1.3061224489795917
.........40 Ocean level= 10.345750146174538  Ocean surface fraction= 1.3061224489795917
.........50 Ocean level= 10.345750146174538  Ocean surface fraction= 1.3112244897959184
.........60 Ocean level= 10.345750146174538  Ocean surface fraction= 1.3265306122448979
.........70 Ocean level= 10.335781188820397  Ocean surface fraction= 1.413265306122449
.........80 Ocean level= 10.332663190011791  Ocean surface fraction= 1.5
.........90 Ocean level= 10.332663190011791 

KeyboardInterrupt: 

In [ ]:
%matplotlib tk

import numpy as np
from landscapeWithOcean import LandscapeWithOcean

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm

def AddHill(Z,NX,NY,xx,yy,r,h):
    for x in range(NX):
        for y in range(NY):
            dx = np.mod(x-xx+NX/2,NX)-NX/2; 
            dy = np.mod(y-yy+NY/2,NY)-NY/2;
            dr = np.sqrt(dx**2+dy**2);
            if (dr<r):
                Z[x,y] += h * (np.cos(dr/r*np.pi/2.0))**2;
    return Z

### Define simulation grid and initial conditions

NX = 140 # Increased number of rows
NY = 140 # Increased number of columns

d  = 5  # grid spacing in meters
dx = d  
dy = d

LX = NX*dx
LY = NY*dy

# Define custom landscape
Z = np.zeros((NX, NY))

# Add custom features
Z = AddHill(Z, NX, NY, NX/4, NY/4, 30, 10)
Z = AddHill(Z, NX, NY, 3*NX/4, 3*NY/4, 20, 15)

x = np.arange(NX)
y = np.arange(NY)
X, Y = np.meshgrid(y, x)
ZMaxOrg = np.max(Z)

### Physical Parameters
K = 5.0e-6  # Increased erosion coefficient
D = 0.01   # Increased diffusion coefficient

# Exponents
n = 2  # Increased sensitivity to steep gradients
m = 1.5  # Increased weight on larger cells

### Ocean Level
oceanLevelParameter = 0.05  # Lower ocean level

# Time step
dt = 90  # Smaller time step

# Initialize landscape 
ls = LandscapeWithOcean(NX,NY)
ls.ComputeOceanVolumeFromOceanLevelParameter(Z, NX, NY, oceanLevelParameter)
ls.pool_check(Z, NX, NY)
ls.A = np.zeros((NX,NY))

# Set-up figure
def init_figure():
    fig = plt.figure(figsize=(12.,6.))
    plt.show()
    return fig

def update_figure():
    plt.clf()
    ax1 = fig.add_subplot(121,projection='3d')

    # use equal x-y aspect with an explicit vertical exaggeration
    vert_exag = 4.
    ax1.set_xlim3d(0, max(NX, NY))
    ax1.set_ylim3d(0, max(NX, NY))
    ax1.set_zlim3d(0, ZMaxOrg)

    ax1.set_title('Surface Relief')

    ZPlot = np.copy(Z)
    ZPlot[ZPlot < ls.ZBeachLevel] = ls.ZBeachLevel 
    ZPlot -= ls.ZBeachLevel
    ax1.plot_surface(X, Y, ZPlot, cmap=cm.terrain, rstride=1, cstride=1, antialiased=False, linewidth=0)

    ax2 = fig.add_subplot(122, aspect='equal')
    ax2.set_title('Elevation')

    im = ax2.pcolor(ZPlot, cmap=cm.coolwarm)
    cs = ax2.contour(ZPlot, 6, colors='k')

    cbar = fig.colorbar(im, shrink=0.5, aspect=5)
    cbar.add_lines(cs)

    plt.draw()
    plt.pause(0.0001)

# Set up figure
fig = init_figure()
update_figure()
Znew = np.copy(Z)

n_iter = 200  # Increased number of iterations

for it in range(1, n_iter+1):
    
    ls.calculate_collection_area(Z, NX, NY)
    ls.A *= dx*dy
    
    for i in range(NX):
        iL = np.mod(i-1, NX)
        iR = np.mod(i+1, NX)

        for j in range(NY):
            jD = np.mod(j-1, NY)
            jU = np.mod(j+1, NY)
  
            if ls.ocean[i, j] > 0:
                Psi_z = 0
                Phi_z = 0
            else:
                if ls.drain[i, j] > 0:
                    s1 = (Z[iR, j] - Z[iL, j]) / (2.0 * dx)
                    s2 = (Z[i, jU] - Z[i, jD]) / (2.0 * dy)
                    s3 = (Z[iR, jD] - Z[iL, jU]) / (2.0 * np.sqrt(dx**2 + dy**2))
                    s4 = (Z[iR, jU] - Z[iL, jD]) / (2.0 * np.sqrt(dx**2 + dy**2))
                    gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2)) / 2.0
                    Psi_z = K * (ls.A[i, j]**m * gradient**n - theta_c)            

                elif ls.drainage[i, j] > 0:
                    if (Z[i, j] >= Z[iR, j]) and ls.pool[iR, j] != ls.drainage[i, j]: 
                        gradient = (Z[i, j] - Z[iR, j]) / dx
                    elif (Z[i, j] >= Z[iL, j]) and ls.pool[iL, j] != ls.drainage[i, j]:
                        gradient = (Z[i, j] - Z[iL, j]) / dx
                    elif (Z[i, j] >= Z[i, jU]) and ls.pool[i, jU] != ls.drainage[i, j]:
                        gradient = (Z[i, j] - Z[i, jU]) / dy
                    elif (Z[i, j] >= Z[i, jD]) and ls.pool[i, jD] != ls.drainage[i, j]:
                        gradient = (Z[i, j] - Z[i, jD]) / dy
                    else:
                        gradient = 0.02
                    Psi_z = K * (ls.A[i, j]**m * gradient**n - theta_c)
                
                else:
                    Psi_z = 0
                
                if (Psi_z < 0):
                    Psi_z = 0 

                Phi_z = D * ((Z[iR, j] - 2.0*Z[i, j] + Z[iL, j]) / dx**2 + (Z[i, jU] - 2.0*Z[i, j] + Z[i, jD]) / dy**2)
           
            Znew[i, j] = Z[i, j] + (Phi_z - Psi_z + uplift) * dt  

            dZdt = (Znew[i, j] - Z[i, j]) / dt
            CFL = abs(dZdt) * dt / min(dx, dy)
            if (CFL > 1.0):
                print('\nWarning: Time step of', dt, 'is probably too large. Safer would be:', dt/CFL)
            
            if (Znew[i, j] < 0.):
                Znew[i, j] = 0.

    Z = np.copy(Znew)
    ls.pool_check(Z, NX, NY)

    if (np.mod(it, 10) == 0): 
        print(it, end='')
        update_figure()
        print(' Ocean level=', ls.ZBeachLevel, ' Ocean surface fraction=', 100*ls.AOcean/(NX*NY))
    else:
        print('.', end='')

update_figure()
print(' Simulation finished.')

In [37]:
%matplotlib tk

import numpy as np
from landscapeWithOcean import LandscapeWithOcean

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import ffmpeg
from matplotlib.animation import FFMpegWriter

def AddHill(Z,NX,NY,xx,yy,r,h):
    for x in range(NX):
        for y in range(NY):
            dx = np.mod(x-xx+NX/2,NX)-NX/2; 
            dy = np.mod(y-yy+NY/2,NY)-NY/2;
            dr = np.sqrt(dx**2+dy**2);
            if (dr<r):
                Z[x,y] += h * (np.cos(dr/r*np.pi/2.0))**2;
    return Z

### Define simulation grid and initial conditions

NX = 140 # Increased number of rows
NY = 140 # Increased number of columns

d  = 5  # grid spacing in meters
dx = d  
dy = d

LX = NX*dx
LY = NY*dy

# Define custom landscape
Z = np.zeros((NX, NY))

# Add custom features
Z = AddHill(Z, NX, NY, NX/4, NY/4, 20, 20)  # Large hill
Z = AddHill(Z, NX, NY, 3*NX/4, 3*NY/4, 15, 10)  # Smaller hill
Z = AddHill(Z, NX, NY, NX/2, NY/2, 10, -15)  # Valley

x = np.arange(NX)
y = np.arange(NY)
X, Y = np.meshgrid(y, x)
ZMaxOrg = np.max(Z)

### Physical Parameters
K = 5.0e-6  # Increased erosion coefficient
D = 0.01   # Increased diffusion coefficient

# Exponents
n = 2  # Increased sensitivity to steep gradients
m = 1.5  # Increased weight on larger cells

### Ocean Level
oceanLevelParameter = 0.05  # Lower ocean level

# Time step
dt = 90  # Smaller time step

# Initialize landscape 
ls = LandscapeWithOcean(NX,NY)
ls.ComputeOceanVolumeFromOceanLevelParameter(Z, NX, NY, oceanLevelParameter)
ls.pool_check(Z, NX, NY)
ls.A = np.zeros((NX,NY))

# Set-up figure
def init_figure():
    fig = plt.figure(figsize=(12.,6.))
    plt.show()
    return fig

def update_figure():
    plt.clf()
    ax1 = fig.add_subplot(121,projection='3d')

    # use equal x-y aspect with an explicit vertical exaggeration
    vert_exag = 4.
    ax1.set_xlim3d(0, max(NX, NY))
    ax1.set_ylim3d(0, max(NX, NY))
    ax1.set_zlim3d(0, ZMaxOrg)

    ax1.set_title('Surface Relief')

    ZPlot = np.copy(Z)
    ZPlot[ZPlot < ls.ZBeachLevel] = ls.ZBeachLevel 
    ZPlot -= ls.ZBeachLevel
    ax1.plot_surface(X, Y, ZPlot, cmap=cm.terrain, rstride=1, cstride=1, antialiased=False, linewidth=0)

    ax2 = fig.add_subplot(122, aspect='equal')
    ax2.set_title('Elevation')

    im = ax2.pcolor(ZPlot, cmap=cm.coolwarm)
    cs = ax2.contour(ZPlot, 6, colors='k')

    cbar = fig.colorbar(im, shrink=0.5, aspect=5)
    cbar.add_lines(cs)

    plt.draw()
    plt.pause(0.0001)

# Set up figure
fig = init_figure()
update_figure()
Znew = np.copy(Z)

n_iter = 100  # Increased number of iterations

for it in range(1, n_iter+1):
    
    ls.calculate_collection_area(Z, NX, NY)
    ls.A *= dx*dy
    
    for i in range(NX):
        iL = np.mod(i-1, NX)
        iR = np.mod(i+1, NX)

        for j in range(NY):
            jD = np.mod(j-1, NY)
            jU = np.mod(j+1, NY)
  
            if ls.ocean[i, j] > 0:
                Psi_z = 0
                Phi_z = 0
            else:
                if ls.drain[i, j] > 0:
                    s1 = (Z[iR, j] - Z[iL, j]) / (2.0 * dx)
                    s2 = (Z[i, jU] - Z[i, jD]) / (2.0 * dy)
                    s3 = (Z[iR, jD] - Z[iL, jU]) / (2.0 * np.sqrt(dx**2 + dy**2))
                    s4 = (Z[iR, jU] - Z[iL, jD]) / (2.0 * np.sqrt(dx**2 + dy**2))
                    gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2)) / 2.0
                    Psi_z = K * (ls.A[i, j]**m * gradient**n - theta_c)            

                elif ls.drainage[i, j] > 0:
                    if (Z[i, j] >= Z[iR, j]) and ls.pool[iR, j] != ls.drainage[i, j]: 
                        gradient = (Z[i, j] - Z[iR, j]) / dx
                    elif (Z[i, j] >= Z[iL, j]) and ls.pool[iL, j] != ls.drainage[i, j]:
                        gradient = (Z[i, j] - Z[iL, j]) / dx
                    elif (Z[i, j] >= Z[i, jU]) and ls.pool[i, jU] != ls.drainage[i, j]:
                        gradient = (Z[i, j] - Z[i, jU]) / dy
                    elif (Z[i, j] >= Z[i, jD]) and ls.pool[i, jD] != ls.drainage[i, j]:
                        gradient = (Z[i, j] - Z[i, jD]) / dy
                    else:
                        gradient = 0.02
                    Psi_z = K * (ls.A[i, j]**m * gradient**n - theta_c)
                
                else:
                    Psi_z = 0
                
                if (Psi_z < 0):
                    Psi_z = 0 

                Phi_z = D * ((Z[iR, j] - 2.0*Z[i, j] + Z[iL, j]) / dx**2 + (Z[i, jU] - 2.0*Z[i, j] + Z[i, jD]) / dy**2 )

            Znew[i, j] = Z[i, j] + (Phi_z - Psi_z + uplift) * dt  

            dZdt = (Znew[i, j] - Z[i, j]) / dt
            CFL = abs(dZdt) * dt / min(dx, dy)
            if (CFL > 1.0):
                print('\nWarning: Time step of', dt, 'is probably too large. Safer would be:', dt/CFL)
            
            if (Znew[i, j] < 0.):
                Znew[i, j] = 0.
    
    Z = np.copy(Znew)
    ls.pool_check(Z, NX, NY)

    if (np.mod(it, 10) == 0): 
        print(it, end='')
        update_figure()
        print(' Ocean level=', ls.ZBeachLevel, ' Ocean surface fraction=', 100*ls.AOcean/(NX*NY));
    else:
        print('.', end='')

update_figure()
print(' Simulation finished.')

Minimum elevation           -15.0
Maximum elevation           20.0
Beach level                 -13.25
Ocean volume                12.639709209341966
Percentage of ocean surface 0.0663265306122449





























.........10 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........20 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........30 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........40 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........50 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........60 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........70 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........80 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........90 Ocean level= 0.000762999168302316  Ocean surface fraction= 

In [38]:
metadata = dict(title='Landscape Evolution', artist='Matplotlib')
writer = FFMpegWriter(fps=15, metadata=metadata,bitrate=200000)
fig = plt.figure(dpi=200)

with writer.saving(fig, "landscape_evolution.mp4", dpi=200):
    for it in range(n_iter):    
        ls.calculate_collection_area(Z, NX, NY)
        ls.A *= dx*dy
        for i in range(NX):
            iL = np.mod(i-1, NX)
            iR = np.mod(i+1, NX)
            for j in range(NY):
                jD = np.mod(j-1, NY)
                jU = np.mod(j+1, NY)
                if ls.ocean[i, j] > 0:
                    Psi_z = 0
                    Phi_z = 0
                else:
                    if ls.drain[i, j] > 0:
                        s1 = (Z[iR, j] - Z[iL, j]) / (2.0 * dx)
                        s2 = (Z[i, jU] - Z[i, jD]) / (2.0 * dy)
                        s3 = (Z[iR, jD] - Z[iL, jU]) / (2.0 * np.sqrt(dx**2 + dy**2))
                        s4 = (Z[iR, jU] - Z[iL, jD]) / (2.0 * np.sqrt(dx**2 + dy**2))
                        gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2)) / 2.0
                        Psi_z = K * (ls.A[i, j]**m * gradient**n - theta_c)            
                    elif ls.drainage[i, j] > 0:
                        if (Z[i, j] >= Z[iR, j]) and ls.pool[iR, j] != ls.drainage[i, j]: 
                            gradient = (Z[i, j] - Z[iR, j]) / dx
                        elif (Z[i, j] >= Z[iL, j]) and ls.pool[iL, j] != ls.drainage[i, j]:
                            gradient = (Z[i, j] - Z[iL, j]) / dx
                        elif (Z[i, j] >= Z[i, jU]) and ls.pool[i, jU] != ls.drainage[i, j]:
                            gradient = (Z[i, j] - Z[i, jU]) / dy
                        elif (Z[i, j] >= Z[i, jD]) and ls.pool[i, jD] != ls.drainage[i, j]:
                            gradient = (Z[i, j] - Z[i, jD]) / dy
                        else:
                            gradient = 0.02
                        Psi_z = K * (ls.A[i, j]**m * gradient**n - theta_c)
                    else:
                        Psi_z = 0
                    if (Psi_z < 0):
                        Psi_z = 0 
                    Phi_z = D * ((Z[iR, j] - 2.0*Z[i, j] + Z[iL, j]) / dx**2 + (Z[i, jU] - 2.0*Z[i, j] + Z[i, jD]) / dy**2 )
                Znew[i, j] = Z[i, j] + (Phi_z - Psi_z + uplift) * dt  
                dZdt = (Znew[i, j] - Z[i, j]) / dt
                CFL = abs(dZdt) * dt / min(dx, dy)
                if (CFL > 1.0):
                    print('\nWarning: Time step of', dt, 'is probably too large. Safer would be:', dt/CFL)
                if (Znew[i, j] < 0.):
                    Znew[i, j] = 0.
        Z = np.copy(Znew)
        ls.pool_check(Z, NX, NY)
        if (np.mod(it, 10) == 0): 
            print(it, end='')
            update_figure()
            writer.grab_frame()
            print(' Ocean level=', ls.ZBeachLevel, ' Ocean surface fraction=', 100*ls.AOcean/(NX*NY));
        else:
            print('.', end='')
    update_figure()
    writer.grab_frame()

0 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........10 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........20 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........30 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........40 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........50 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........60 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........70 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........80 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........90 Ocean level= 0.000762999168302316  Ocean surface fraction= 89.4438775510204
.........